# Week 2

**`Agent::API::base`**
* `fit` method
* private API (agent-specific)
* public API (environment runner friendly)

**`Environment::API::runner`**
* fixed agent API
* multiple episodes

In [1]:
# change current working directory
%cd ..

/Users/alpha/Developer/qtrader


In [2]:
# suppress warning messages
import warnings
warnings.filterwarnings('ignore')

# trading environment
from qtrader.envs import TradingEnv
# machine floating-point relative accuracy
from qtrader.utils.numpy import eps

# YAML parser
import yaml

# # scientific programming
# import numpy as np
# import pandas as pd

# # visualization
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
# fetch configuration file
config = yaml.load(open('config/log/week_2.yaml', 'r'))

# configuration summary
print(f"start date: {config['start_date']}")
print(f"trading frequency: {config['freq']}")
print(f"trading universe: {config['tickers']}")

start date: 2015-01-01
trading frequency: W-FRI
trading universe: ['AAPL', 'GE', 'JPM', 'MSFT', 'VOD', 'GS', 'TSLA', 'MMM']


### Trading Environment

Initialize a trading environment, with which the agents will interact.

In [4]:
# initialize environment
env = TradingEnv(universe=config['tickers'],
                 trading_period=config['freq'],
                 csv=config['csv_file_prices'])